# Inference

## Eric He

## Homework 2 - Problem 5

In problem 5, we are asked to provide code to build a Hidden Markov Model and train it on the sequence in `sequence.txt`.

We can find the parameters of the Hidden Markov Model using the Baum-Welch algorithm, an expectation-maximization style algorithm that works as follows:

# Baum-Welch algorithm

## Expectation: Given model parameters, compute the probability of the data

This probability, when instead viewed as a function of the model parameters, is the likelihood function. 

## Maximization: Given the likelihood function 

# Import necessary packages and read in sequence.txt

I've changed `sequence.txt` into a Python file and imported that instead.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style('whitegrid')

import sequence
s = sequence.sequence

# 

In [16]:
N = 2
A = np.random.rand(N, N)

In [19]:
A 

array([[0.9996077 , 0.67353356],
       [0.16476904, 0.88172843]])

In [37]:
A.transpose()

array([[0.9996077 , 0.16476904],
       [0.67353356, 0.88172843]])

In [24]:
A.sum(axis=1)

array([1.67314126, 1.04649747])

In [50]:
P = np.array([0.5, 0.5])
B = np.random.rand(2, 5)
Y = [y-1 for y in s]
P * B[:,Y[0]]

array([0.11484408, 0.22066467])

In [52]:
P

array([0.5, 0.5])

In [53]:
B[:, Y[0]]

array([0.22968817, 0.44132933])

In [54]:
B

array([[0.36482084, 0.9357858 , 0.84127471, 0.22968817, 0.59340239],
       [0.06158862, 0.75407686, 0.28255176, 0.44132933, 0.76552895]])

In [57]:
for i in range(1, 3):
    print(i)

1
2


In [400]:
class HMM():
    def __init__(self, N, Y):
        # N is the number of hidden nodes
        self.N = N
        self.i = None
        if Y is not None:
            ### initialize params ###
            # store the data; map the input to easy 0-based index
            self.Y = [y - 1 for y in Y]
            # T is the number of observations
            self.T = len(Y)
            # K is the number of observable states
            self.K = len(set(self.Y))
            # A is the matrix of hidden state transition probabilities
            # random initialization
            self.A = np.random.rand(self.N, self.N)
            self.A = (self.A.transpose() / self.A.sum(axis=1)).transpose()
            # B is the estimated probabilities for each observable state given a hidden state
            # random initialization
            self.B = np.random.rand(self.N, self.K)
            self.B = (self.B.transpose() / self.B.sum(axis=1)).transpose()
            # P is the initial probability of the hidden states
            # uniform initialization
            self.P = np.ones(self.N) / N

            self.previous_A = self.A.copy()
            self.previous_B = self.B.copy()
        else:
            self.Y = None
    
    def fit(self, Y, max_iter=10):
        ### initialize params ###
        # store the data; map the input to easy 0-based index
        self.Y = [y - 1 for y in Y]
        # T is the number of observations
        self.T = len(Y)
        # K is the number of observable states
        self.K = len(set(self.Y))
        # A is the matrix of hidden state transition probabilities
        # random initialization
        self.A = np.random.rand(self.N, self.N)
        self.A = (self.A.transpose() / self.A.sum(axis=1)).transpose()
        # B is the estimated probabilities for each observable state given a hidden state
        # random initialization
        self.B = np.random.rand(self.N, self.K)
        self.B = (self.B.transpose() / self.B.sum(axis=1)).transpose()
        # P is the initial probability of the hidden states
        # uniform initialization
        self.P = np.ones(self.N) / N
        
        self.previous_A = self.A.copy()
        self.previous_B = self.B.copy()
        
        # expectation maximization loop
        while self.convergence_check(max_iter=max_iter):
            alphas, betas = self.expectation()
            self.maximization(alphas, betas)
    
    def forward(self):
        """Computes alphas, the probability of seeing observations y1...yt and having state i at time t"""
        alphas = np.zeros([self.N, self.T])
        for n in range(self.N):
            alphas[:, 0] = self.P * self.B[:,self.Y[0]]
        alphas[:, 0] = alphas[:, 0] / alphas[:, 0].sum()
            
        for t in range(1, self.T):
            alphas[:, t] = self.B[:,self.Y[t]] * (alphas[:,t-1] @ self.A)
            alphas[:, t] = alphas[:, t] / alphas[:, t].sum()
            
        return alphas

    def backward2(self):
        betas = np.ones([self.N, self.T]) / 2
        
        for t in reversed(range(self.T - 1)):
            for n in range(self.N):
                betas[n, t] = np.sum(betas[:,t+1] * self.A[n,:] * self.B[n, Y[t+1]])
            betas[:, t] = betas[:, t] / betas[:, t].sum()
            
        return betas
    
    def backward(self):
        betas = np.ones([self.N, self.T]) / 2
        
        for t in reversed(range(self.T - 1)):
            betas[:, t] = self.A @ (self.B[:,self.Y[t+1]] * betas[:,t+1])
            betas[:, t] = betas[:, t] / betas[:, t].sum()
            
        return betas
    
    # check 
    def convergence_check(self, max_iter=10):
        if self.i is None:
            self.i = 1
            return True
        if self.i > max_iter:
            return False
        else:
            self.i += 1
            log_likelihood = np.sum(np.log(gammas.T @ self.B))
            print('Log-likelihood is {}'.format(np.log(hmm.gammas.T @ hmm.B).sum()))
            print('Difference in A: {}'.format(np.abs(self.previous_A - self.A).sum()))
            print('Difference in B: {}'.format(np.abs(self.previous_B - self.B).sum()))
            print('\n')
            self.previous_A = self.A
            self.previous_B = self.B
            
            return True
    
    def expectation(self):
        alphas = self.forward()
        betas = self.backward()
        return alphas, betas
    
    def maximization(self, alphas, betas):
        gammas = alphas * betas
        gammas = gammas / gammas.sum(axis=0)
        
        xis = np.zeros([self.N, self.N, self.T])
        for t in range(self.T - 1):
            for i in range(self.N):
                for j in range(self.N):
                    xis[i, j, t] = alphas[i, t] * self.A[i,j] * self.B[j,self.Y[t+1]]
                    # there should be a normalization here
            xis[:,:,t] = (xis[:,:,0].T / xis[:,:,0].sum()).T            
            
        self.P = gammas[:,0]
        
        self.A = xis[:,:,:self.T-1].sum(axis=2) / gammas[:,:self.T-1].sum(axis=1)
        self.A = (self.A.transpose() / self.A.sum(axis=1)).transpose() # QUESTIONABLE AND SHOULDNT BE NEEDED
        
        self.xis = xis
        self.gammas = gammas
        
        hidden_and_observed = pd.DataFrame(gammas.T)
        hidden_and_observed['Y'] = self.Y
        empirical_observed = hidden_and_observed.groupby('Y').sum()
        self.B = (empirical_observed.T / empirical_observed.sum(axis=1).T).values

In [402]:
hmm = HMM(2,Y)
hmm.fit(Y)

Log-likelihood is -21204.40821963038
Difference in A: 1.3106934725884325
Difference in B: 4.08670835181781


Log-likelihood is -10180.253276069128
Difference in A: 0.6923642526968634
Difference in B: 3.6335644953906643


Log-likelihood is -7144.675120207471
Difference in A: 0.06453008418343098
Difference in B: 1.0358769378319066


Log-likelihood is -5474.396686365913
Difference in A: 0.3389493352069977
Difference in B: 0.7576209798664649


Log-likelihood is -5855.464413507135
Difference in A: 0.6107506480652101
Difference in B: 0.686915394500848


Log-likelihood is -9497.081100134816
Difference in A: 0.11516148780173824
Difference in B: 1.3024467494378411


Log-likelihood is -12593.170780864228
Difference in A: 1.781576742327836
Difference in B: 1.7093360867021414


Log-likelihood is -2273.981516486681
Difference in A: 0.7635985721288225
Difference in B: 2.2423789178846554


Log-likelihood is -1358.6742851618492
Difference in A: 0.09668874742306646
Difference in B: 0.2535919622083581



In [390]:
hmm.B.shape

(2, 3)

In [399]:
alphas = np.zeros([hmm.N, hmm.T])
for n in range(hmm.N):
    alphas[:, 0] = hmm.P * hmm.B[:,hmm.Y[0]]
alphas[:, 0] = alphas[:, 0] / alphas[:, 0].sum()

for t in range(1, hmm.T):
    alphas[:, t] = hmm.B[:,self.Y[t]] * (alphas[:,t-1] @ hmm.A)
    alphas[:, t] = alphas[:, t] / alphas[:, t].sum()

NameError: name 'self' is not defined

In [397]:
hmm.B[:,Y[t]]

IndexError: index 3 is out of bounds for axis 1 with size 3

In [398]:
Y[t]

3

In [404]:
hmm = HMM(2, Y = [3,3,1,1,2,2,3,1,3])
hmm.A = transition
hmm.B = emission
hmm.P = [0.5, 0.5]

pd.DataFrame(hmm.forward())

,0,1,2,3,4,5,6,7,8
0,0.875,0.963768,0.46886,0.114775,0.200381,0.266963,0.766093,0.25631,0.759137
1,0.125,0.036232,0.53114,0.885225,0.799619,0.733037,0.233907,0.74369,0.240863


In [405]:
pd.DataFrame(hmm.backward())

,0,1,2,3,4,5,6,7,8
0,0.541986,0.150698,0.272933,0.647765,0.689984,0.744265,0.384774,0.791667,0.5
1,0.458014,0.849302,0.727067,0.352235,0.310016,0.255735,0.615226,0.208333,0.5


In [417]:
_ = forward_probs()
pd.DataFrame(_[0] / _[0].sum(axis=0))

,0,1,2,3,4,5,6,7,8
0,0.875,0.963768,0.46886,0.114775,0.200381,0.266963,0.766093,0.25631,0.759137
1,0.125,0.036232,0.53114,0.885225,0.799619,0.733037,0.233907,0.74369,0.240863


In [416]:
__ = backward_probs()
pd.DataFrame(__[0] / __[0].sum(axis=0))

,0,1,2,3,4,5,6,7,8
0,0.541986,0.150698,0.272933,0.647765,0.689984,0.744265,0.384774,0.791667,0.5
1,0.458014,0.849302,0.727067,0.352235,0.310016,0.255735,0.615226,0.208333,0.5


# Implementation below is swiped from https://github.com/hamzarawal/HMM-Baum-Welch-Algorithm/blob/master/baum-welch.py

In [377]:
#generating initial probabilities

#transition probabilities
transition = np.array([[0.8,0.1],
                       [0.1,0.8]])
#Emission probabilities
emission = np.array([[0.1,0.2,0.7],
                     [0.7,0.2,0.1]])

#defining states and sequence symbols
states = ['H','C']
states_dic = {'H':0, 'C':1}
sequence_syms = {'1':0,'2':1,'3':2}
sequence = ['1','2','3']

#test sequence
test_sequence = '331122313'
test_sequence = [x for x in test_sequence]

#probabilities of going to end state
end_probs = [0.1, 0.1]
#probabilities of going from start state
start_probs = [0.5, 0.5]

#function to find forward probabilities
def forward_probs():
    # node values stored during forward algorithm
    node_values_fwd = np.zeros((len(states), len(test_sequence)))

    for i, sequence_val in enumerate(test_sequence):
        for j in range(len(states)):
            # if first sequence value then do this
            if (i == 0):
                node_values_fwd[j, i] = start_probs[j] * emission[j, sequence_syms[sequence_val]]
            # else perform this
            else:
                values = [node_values_fwd[k, i - 1] * emission[j, sequence_syms[sequence_val]] * transition[k, j] for k in
                          range(len(states))]
                node_values_fwd[j, i] = sum(values)

    #end state value
    end_state = np.multiply(node_values_fwd[:, -1], end_probs)
    end_state_val = sum(end_state)
    return node_values_fwd, end_state_val

In [366]:
#function to find backward probabilities
def backward_probs():
    # node values stored during forward algorithm
    node_values_bwd = np.zeros((len(states), len(test_sequence)))

    #for i, sequence_val in enumerate(test_sequence):
    for i in range(1,len(test_sequence)+1):
        for j in range(len(states)):
            # if first sequence value then do this
            if (-i == -1):
                node_values_bwd[j, -i] = end_probs[j]
            # else perform this
            else:
                values = [node_values_bwd[k, -i+1] * emission[k, sequence_syms[test_sequence[-i+1]]] * transition[j, k] for k in range(len(states))]
                node_values_bwd[j, -i] = sum(values)

    #start state value
    start_state = [node_values_bwd[m,0] * emission[m, sequence_syms[test_sequence[0]]] for m in range(len(states))]
    start_state = np.multiply(start_state, start_probs)
    start_state_val = sum(start_state)
    return node_values_bwd, start_state_val


#function to find si probabilities
def si_probs(forward, backward, forward_val):

    si_probabilities = np.zeros((len(states), len(test_sequence)-1, len(states)))

    for i in range(len(test_sequence)-1):
        for j in range(len(states)):
            for k in range(len(states)):
                si_probabilities[j,i,k] = ( forward[j,i] * backward[k,i+1] * transition[j,k] * emission[k,sequence_syms[test_sequence[i+1]]] ) \
                                                    / forward_val
    return si_probabilities

#function to find gamma probabilities
def gamma_probs(forward, backward, forward_val):

    gamma_probabilities = np.zeros((len(states), len(test_sequence)))

    for i in range(len(test_sequence)):
        for j in range(len(states)):
            #gamma_probabilities[j,i] = ( forward[j,i] * backward[j,i] * emission[j,sequence_syms[test_sequence[i]]] ) / forward_val
            gamma_probabilities[j, i] = (forward[j, i] * backward[j, i]) / forward_val

    return gamma_probabilities

In [352]:
baum_welch(2, 6, Y)

ValueError: operands could not be broadcast together with shapes (2,6) (2,) 

In [351]:
set(s)

{1, 2, 3, 4, 5, 6}

In [353]:
num_obs = 25
observations1 = np.random.randn( num_obs )
observations1[observations1>0] = 1
observations1[observations1<=0] = 0

In [356]:
A_mat, O_mat = baum_welch(2,2,observations1)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices